In [2]:
#Importing the libraries that will be used further

from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.feature_extraction import image
import numpy as np
import cv2


In [3]:
#Reading the input image

img = cv2.imread('2.jpg')

row,col,no_channel = img.shape

#Getting the blue,green and red channel(2d array) from img
b,g,r = cv2.split(img)

#Window size(As specified in the paper) to slide across image for variance calculation
window_shape = (6, 6)

In [6]:
#sklearn library to extract all 2d patches of specified window shape from each channel

blue_patches = extract_patches_2d(b, window_shape)
green_patches = extract_patches_2d(g, window_shape)
red_patches = extract_patches_2d(r, window_shape)


/home/abhishek/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/image.py:287: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  indexing_strides = arr[slices].strides


In [42]:
# Variables
Dg = []
Db = []

Bl_pixels = []
Gr_pixels = []
Re_pixels = []

#Estimating the pixels whose patch variation is < 0.9

for i in range(len(blue_patches)):
    bl = blue_patches[i]
    gr = green_patches[i]
    re = red_patches[i]
    
    bls = list(bl.flat)
    grls = list(gr.flat)
    rels = list(re.flat)
    
    if np.var(bl) < 0.9 or np.var(gr) < 0.9 or np.var(re) < 0.9:
        Db.extend([a - b for a, b in zip(bls, rels)])
        Dg.extend([a - b for a, b in zip(grls, rels)])

        Bl_pixels.extend(bls)
        Gr_pixels.extend(grls)
        Re_pixels.extend(rels)

#Getting the max of Db and Dg

maxB = Db[0]
maxG = Dg[0]
indB = 0
indG = 0

ind = 0
maxR = 0
for i in range(1,len(Db)):
    if maxB < Db[i] or maxG < Dg[i]:
        if maxR < Re_pixels[i]:
            ind = i
            maxB = Db[i]
            maxG = Dg[i]
            maxR = Re_pixels[i]

#Backlight Pixels
Backlight = []

Backlight.append(Bl_pixels[ind])
Backlight.append(Gr_pixels[ind])
Backlight.append(Re_pixels[ind])

normBacklight = np.divide(Backlight,255)


In [44]:
Backlight, normBacklight

([150, 176, 86], array([0.58823529, 0.69019608, 0.3372549 ]))

In [45]:
#Finding attenuation ratios, formula's  is taken from the paper.

Sb = (-0.00113*450 + 1.6251)
Sg = (-0.00113*540 + 1.6251)
Sr = (-0.00113*620 + 1.6251)

#Attenuation ratios

lamda_b = (Sb*Backlight[2])/(Sr*Backlight[0])

lamda_g = (Sg*Backlight[2])/(Sr*Backlight[1])

print('lamda_b : ',lamda_b, ' lamda_g : ',lamda_g)

lamda_b :  0.6924651162790698  lamda_g :  0.536416490486258


In [183]:
#Transmission Estimation : 
#i) Finding Attenuation curves.(Clustering , building KD Tree)

#Normalize the input image to double precision.
norm_img = cv2.normalize(img.astype('double'), None, 0.0, 1.0, cv2.NORM_MINMAX)


#finding haze lines
dist_from_backlight = np.zeros([row,col,no_channel],dtype = np.double)

for i in range(no_channel):
    dist_from_backlight[:,:,i] = norm_img[:,:,i]-normBacklight[i]

    
#calculating radius
radius = np.sqrt((dist_from_backlight[:,:,0])**2 + (dist_from_backlight[:,:,1])**2 +(dist_from_backlight[:,:,2])**2 )

#clustering the pixels
dist_unit_radius = np.reshape(dist_from_backlight,(row*col,no_channel))
dist_norm = np.sqrt(np.sum(dist_unit_radius**2,axis=0))
dist_unit_radius = np.divide(dist_unit_radius,dist_norm)
n_points = 1000

#read the points from file
file = open("two.txt","r")
d = np.loadtxt(file)
points = d.reshape(1000,3)

file.close()

from sklearn.neighbors import KDTree
kdt = KDTree(points, leaf_size=50, metric='euclidean')

nearest_dist, nearest_ind = kdt.query(dist_unit_radius, k=1)

In [184]:
nearest_dist.shape,nearest_ind.shape

((538900, 1), (538900, 1))

In [190]:
# ii) Estimating Initial Relative Transmission

#K = accumarray(ind,radius(:),[n_points,1],@max);

K = np.zeros((n_points,1),dtype = np.double)

uniqNearest_Ind = np.unique(nearest_ind)

temp = radius.flatten()

for item in uniqNearest_Ind:
    indexes = np.where(nearest_ind == item)[0]
    K[item] = np.max(temp[indexes])

K_new = K[nearest_ind]

radius_new = np.reshape(K_new,(row,col))

transmission_estimation = np.divide(radius,radius_new)

#Limit the transmission to the range [trans_min, 1] for numerical stability
trans_min = 0.1;
transmission_estimation[np.where(transmission_estimation == 0.0)[0]] = 0.1
transmission_estimation[np.where(transmission_estimation > 1)[0]] = 1


In [199]:
# iii) Refining the transmission using WLS Optimization

# find bin counts for reliability - small bins (#pixels<50) do not comply with 
# the model assumptions and should be disregarded

bin_count = np.zeros((1000,1),dtype=int)

uniqNearest_Ind = np.unique(nearest_ind)

for item in uniqNearest_Ind:
    bin_count[item] = np.sum(nearest_ind[:] == item)

bin_count_map = bin_count[nearest_ind]

bin_count_map = bin_count_map.reshape((row,col))

bin_count_map.shape



(634, 850)

In [200]:
def bin_eval_fun(x):
    res = np.zeros((x.shape[0],x.shape[1]))
    
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            res[i,j] = min(1,x[i,j]/50)
            
    return res

In [201]:
#Calculate std - this is the data-term weight

K_std = np.zeros((n_points,1))

radius_flat = radius.flatten()

uniqRadius = np.unique(radius_flat)

for item in uniqNearest_Ind:
    indexes = np.where(nearest_ind == item)[0]
    
    K_std[item] = np.std(radius_flat[indexes])

radius_std = K_std[nearest_ind]

radius_std = radius_std.reshape((row,col))

In [202]:
def radius_eval_fun(r):
    res = np.zeros((r.shape[0],r.shape[1]))
    
    for i in range(r.shape[0]):
        for j in range(r.shape[1]):
            res[i,j] = min(1,3*max(0.001,r[i,j] - 0.1))

    
    return res


In [203]:
radius_reliability = radius_eval_fun(np.divide(radius_std,max(radius_std.flatten())))

data_term_weight = bin_eval_fun(bin_count_map) * radius_reliability

In [204]:
# WLS Optimization
# Refered from Github : https://github.com/danaberman/non-local-dehazing (modified)

r_trans = np.array(transmission_estimation)

L = np.log(r_trans)

lamda = 0.05
smallNum = 0.00001

r,c = r_trans.shape

k = r*c

dy = np.diff(L,0,0)

old_dy = dy
absDy = abs(dy)

power = (np.power(absDy,2) + smallNum)       #Github Impl

dy = np.divide(lamda,power)

dy = np.pad(dy,(1,0),'constant')

dy = dy.flatten()

In [205]:
dx = np.diff(L,0,1)

old_dx = dx

absDx = abs(dx)

powerDx = (np.power(absDx,2) + smallNum)

dx = np.divide(lamda,powerDx)

dx = np.pad(dx,(0,1),'constant')

dx = dx.flatten()

In [206]:
lapMat = np.zeros(shape=(len(dx),2))
lapMat[:,0] = dx
lapMat[:,1] = dy

d = [-r,-1]

lapMat = np.transpose(lapMat)

import scipy

reslapMat = scipy.sparse.spdiags(lapMat,d,k,k)


In [207]:
reslapMat.shape

(538900, 538900)

In [208]:
e = dx

w = np.lib.pad(dx,(r,0),'constant',constant_values=(0))

w = w[0:len(w)-r]

s = dy

n = np.lib.pad(dy,(1,0),'constant',constant_values=(0))

n = n[0:(len(n) - 1)]

D = 1-(e+w+s+n)

A = reslapMat

temp = np.transpose(D)

A = A + np.transpose(A) + scipy.sparse.spdiags(temp, 0, k, k);



In [209]:
#Normalize data weight

data_term_weight = data_term_weight - min(data_term_weight.flatten())

data_term_weight = np.divide(data_term_weight,max(data_term_weight.flatten()) + smallNum)



In [210]:
reliability_mask = data_term_weight[0,:] < 0.6

in_row1 = np.zeros((r_trans.shape[1]))

for i in range(r_trans.shape[1]):
        in_row1[i] = np.min(r_trans[:,i])

data_term_weight[0,reliability_mask] = 0.8
r_trans[0,reliability_mask] = in_row1[reliability_mask]

Adata = scipy.sparse.spdiags(data_term_weight.flatten(), 0, k, k);


In [211]:
Res1 = Adata + A
Res2 = Adata*(r_trans.flatten())

from scipy.sparse.linalg import spsolve

x = spsolve(Res1,Res2)      

In [212]:
x.shape,x

((538900,), array([ 0.26586673,  0.32570625,  0.33027653, ...,  0.71264693,
        -0.08683007,  0.641879  ]))

In [213]:
refined_trans = x.reshape((r,c))

In [217]:
#normBacklight.extend([(1-item) for item in normBacklight])

newBacklight = np.array([normBacklight[0],normBacklight[1],normBacklight[2],1-normBacklight[0],1-normBacklight[1],1-normBacklight[2]])

denoMax = max(newBacklight)


In [262]:

#Estimating K

K_list = []

t_b,t_g,t_r = cv2.split(img)

for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        tempLS = []
        tempLS.append(t_b[i][j]/256 - normBacklight[0])
        tempLS.append(t_g[i][j]/256 - normBacklight[1])
        tempLS.append(t_r[i][j]/256 - normBacklight[2])
        K_list.append(max(tempLS)/denoMax)

K = max(K_list)

In [311]:
normB = t_b/255
normG = t_g/255
normR = t_r/255
normImg = img/255

lambda_r = max(lamda_b,lamda_g)

#Obaining the scene radiance

J_b = np.zeros((img.shape[0],img.shape[1]))
J_g = np.zeros((img.shape[0],img.shape[1]))
J_r = np.zeros((img.shape[0],img.shape[1]))

for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        J_b[i,j] = (((normB[i,j] - normBacklight[0])/(max((K*(refined_trans[i,j]**lamda_b)),0.1))) + normBacklight[0])
        J_g[i,j] = (((normG[i,j] - normBacklight[1])/(max((K*(refined_trans[i,j]**lamda_g)),0.1))) + normBacklight[1])
        J_r[i,j] = (((normR[i,j] - normBacklight[2])/(max((K*(refined_trans[i,j]**lambda_r)),0.1))) + normBacklight[2])
        
        if np.isnan(J_b[i,j]) or J_b[i,j] < 0:
            J_b[i,j] = normBacklight[0]
        if np.isnan(J_g[i,j]) or J_g[i,j] < 0:
            J_g[i,j] = normBacklight[1]
        if np.isnan(J_r[i,j]) or J_r[i,j] < 0:
            J_r[i,j] = normBacklight[2]

/home/abhishek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/abhishek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/home/abhishek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


In [290]:
np.power(refined_trans[0,674],lambda_r) + 1

/home/abhishek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in power
  """Entry point for launching an IPython kernel.


nan

In [313]:
J_b = J_b*255
J_g = J_g*255
J_r = J_r*255

In [314]:
outputImg = np.zeros((img.shape[0],img.shape[1],3),dtype=np.uint8)

outputImg[:,:,0] = J_b
outputImg[:,:,1] = J_g
outputImg[:,:,2] = J_r

cv2.imwrite('Try3Output6.png',outputImg)

True

In [294]:
np.sum(np.isnan(refined_trans))

0

In [295]:
K

0.8583540482954545

In [296]:
normBacklight

array([0.58823529, 0.69019608, 0.3372549 ])

In [265]:
normBacklight[0]

0.5882352941176471

In [275]:
np.sum(np.where(normR==  0.0))

0

In [297]:
lambda_r,lamda_b,lamda_g

(0.6924651162790698, 0.6924651162790698, 0.536416490486258)

In [304]:
np.sum(np.isnan(J_b))

0

In [312]:
np.sum(J_b < 0)

0